In [26]:
import pandas as pd
import numpy as np, warnings
from pathlib import Path
from importlib import reload

# os.chdir('D:/Kha/UWA/CITS5553/sepsis_prediction/data-science-capstone-project/Kha Huynh')
import utils
reload(utils)

np.warnings = warnings
# os.chdir('D:/Kha/UWA/CITS5553/sepsis_prediction/data-science-capstone-project')

ROOT_DIR = Path('../..')

In [27]:
import DataLoader as DataLoader
reload(DataLoader)

dataLoader = DataLoader.DataLoader(ROOT_DIR)

df_desc_icd, df_desc_labitems, df_desc_items = dataLoader.load_descriptions()
df_diagnoses_icd = dataLoader.load_diagnoses_icd(df_desc_icd)
df_demographic = dataLoader.load_demographic(df_diagnoses_icd)
df_demographic = dataLoader.demographic_clean_AGE(df_demographic)
# df_labevents = dataLoader.load_labevents(df_demographic)

Removed 8210 admissions with AGE < 18


# Common Comorbidity

In [28]:
sepsis_cases = df_diagnoses_icd[df_diagnoses_icd.IS_SEPSIS == 1].drop_duplicates(['SUBJECT_ID', 'HADM_ID'])[['SUBJECT_ID', 'HADM_ID']]
sepsis_cases = sepsis_cases.merge(df_diagnoses_icd[df_diagnoses_icd.IS_SEPSIS != 1][['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']], on=['SUBJECT_ID', 'HADM_ID'])
sepsis_cases = sepsis_cases.merge(df_desc_icd[['ICD9_CODE', 'SHORT_TITLE']], on='ICD9_CODE')
sepsis_cases = sepsis_cases.merge(df_demographic[['SUBJECT_ID', 'HADM_ID']], on=['SUBJECT_ID', 'HADM_ID'])
common_icd = sepsis_cases.groupby(['ICD9_CODE', 'SHORT_TITLE']).apply(lambda x: len(x.drop_duplicates(['SUBJECT_ID', 'HADM_ID']))).sort_values(ascending=False).reset_index(name='SEPSIS_COUNT')

common_icd['SEPSIS_percentage'] = common_icd['SEPSIS_COUNT']*100/df_demographic[df_demographic.IS_SEPSIS==1].shape[0] # 5176

In [29]:
nonsepsis_cases = df_demographic[df_demographic.IS_SEPSIS == 0].drop_duplicates(['SUBJECT_ID', 'HADM_ID'])[['SUBJECT_ID', 'HADM_ID']]
nonsepsis_cases = nonsepsis_cases.merge(df_diagnoses_icd[(df_diagnoses_icd.IS_SEPSIS != 1) & (df_diagnoses_icd.ICD9_CODE.isin(common_icd.ICD9_CODE))][['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']], on=['SUBJECT_ID', 'HADM_ID'])
nonsepsis_cases = nonsepsis_cases.merge(df_desc_icd[['ICD9_CODE', 'SHORT_TITLE']], on='ICD9_CODE')
# sepsis_cases = nonsepsis_cases.merge(df_demographic[['SUBJECT_ID', 'HADM_ID']], on=['SUBJECT_ID', 'HADM_ID'])
common_icd_nonsepsis = nonsepsis_cases.groupby(['ICD9_CODE', 'SHORT_TITLE']).apply(lambda x: len(x.drop_duplicates(['SUBJECT_ID', 'HADM_ID']))).sort_values(ascending=False).reset_index(name='NONSEPSIS_COUNT')
common_icd_nonsepsis['NON_SEPSIS_percentage'] = common_icd_nonsepsis['NONSEPSIS_COUNT']*100/df_demographic[df_demographic.IS_SEPSIS==0].shape[0] # 5176

common_icd = common_icd.merge(common_icd_nonsepsis[['ICD9_CODE', 'NONSEPSIS_COUNT', 'NON_SEPSIS_percentage']], on='ICD9_CODE')

# utils.save_csv(common_icd, ROOT_DIR / 'data/report_commorbs.csv')

In [30]:
common_icd.head(50)

,ICD9_CODE,SHORT_TITLE,SEPSIS_COUNT,SEPSIS_percentage,NONSEPSIS_COUNT,NON_SEPSIS_percentage
0,0389,Septicemia NOS,3102,59.930448,604,1.324852
1,78552,Septic shock,2442,47.179289,143,0.313665
2,5849,Acute kidney failure NOS,2106,40.687790,7006,15.367405
3,51881,Acute respiratry failure,1963,37.925039,5529,12.127660
4,4019,Hypertension NOS,1750,33.809892,18932,41.526651
5,4280,CHF NOS,1745,33.713292,11352,24.900197
6,42731,Atrial fibrillation,1605,31.008501,11281,24.744462
7,5990,Urin tract infection NOS,1329,25.676198,5218,11.445492
8,2762,Acidosis,1168,22.565688,3343,7.332748
9,25000,DMII wo cmp nt st uncntr,1069,20.653014,7988,17.521386


## Selected Features

In [31]:
pontential_events = pd.read_csv(ROOT_DIR / 'data/labevents_comparison_sorted.csv')
pontential_events['SEPSIS_PROP'] = pontential_events['Sepsis_abnormal']  * 100/ df_demographic[df_demographic.IS_SEPSIS==1].shape[0]

nonsepsis_size = df_demographic[df_demographic.IS_SEPSIS==0].shape[0]
pontential_events['NonSepsis_abnormal'] = pontential_events['Abnormal_Count'] - pontential_events['Sepsis_abnormal']
pontential_events['NONSEPSIS_PROP'] = pontential_events['NonSepsis_abnormal'] * 100/ nonsepsis_size

pontential_events = pontential_events.drop(['Admission_proportion', 'Abnormal_Count', 'Sepsis_proportion', 'Sepsis_admission_proportion'], axis=1).sort_values('Sepsis_abnormal', ascending=False)

In [32]:
current_using_events = pd.read_csv(ROOT_DIR / 'data/potential_events.csv', nrows=40)
current_using_events = current_using_events[['ITEMID']].merge(pontential_events, on='ITEMID')
# utils.save_csv(current_using_events, ROOT_DIR / 'data/report_features.csv')

In [33]:
current_using_events

,ITEMID,LABEL,FLUID,ref1,ref2,chatgpt,neo4j,Sepsis_abnormal,SEPSIS_PROP,NonSepsis_abnormal,NONSEPSIS_PROP
0,51279,RED BLOOD CELLS,BLOOD,no,no,no,yes,5109,98.705564,43107,94.553630
1,51222,HEMOGLOBIN,BLOOD,yes,no,yes,yes,5114,98.802164,42535,93.298969
2,51221,HEMATOCRIT,BLOOD,yes,no,no,yes,5094,98.415765,42724,93.713534
3,50931,GLUCOSE,BLOOD,no,yes,no,yes,5108,98.686244,43091,94.518535
4,51244,LYMPHOCYTES,BLOOD,no,yes,no,yes,4811,92.948223,25470,55.867515
5,51256,NEUTROPHILS,BLOOD,no,yes,yes,yes,4827,93.257342,27690,60.737004
6,51006,UREA NITROGEN,BLOOD,no,no,yes,yes,4663,90.088872,31697,69.526212
7,51301,WHITE BLOOD CELLS,BLOOD,no,no,yes,yes,4824,93.199382,34151,74.908971
8,51274,PT,BLOOD,no,no,yes,yes,4703,90.861669,32398,71.063830
9,50912,CREATININE,BLOOD,yes,yes,yes,yes,4154,80.255023,21021,46.108796
